Capeet Giglist
===

Filtered to "Wien"

In [262]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime


html_string = requests.get(url).text

soup = BeautifulSoup(html_string, "html.parser")
text_content = soup.get_text().split("""\n\n\n\nKW""")[1]
text_content = [i for i in text_content.splitlines() if (i != "") & (", Wien" in i)]
df = pd.DataFrame({"Text": text_content})
df = df.Text.str.split("\.:\s", n=1, regex=True, expand=True)
df = df.dropna(axis=0, how="any")
df.columns = ["Date", "Text"]
df["Date"] = df["Date"].str.replace("\t", "")
df[["Day", "Month"]] = df["Date"].str.split(".", n=1, expand=True)
df["Day"] = df["Day"].astype(int)
df["Month"] = df["Month"].astype(int)
df = df.drop("Date", axis=1)

df[["Event", "Location"]] = df["Text"].str.split("@", n=1, expand=True)
df["Location"] = df["Location"].str.replace("\[fb\]$", "", regex=True)
df["Location"] = df["Location"].str.replace(", Wien", "", regex=False)

df = df.drop("Text", axis=1)

year_to_add = datetime.now().year
previous_month = datetime.now().month


def add_year(month):
    global year_to_add, previous_month

    if previous_month > month:
        year_to_add += 1
    previous_month = month

    return year_to_add


df["Year"] = df["Month"].apply(add_year)
df["Date"] = df[["Year", "Month", "Day"]].apply(
    lambda x: pd.Timestamp(x["Year"], x["Month"], x["Day"]), axis=1
)
df = df.drop(["Day", "Month", "Year"], axis=1)
df = df.reindex(columns=["Date", "Event", "Location"])

df

,Date,Event,Location
0,2023-10-05,CHALK HANDS (uk) / CLUTCH AT STRAWS,Arena-Beisl
1,2023-10-05,CHOROSIA / THOSAR / GRIMMS EYE,Viper Room
2,2023-10-05,FISH BASKET (pol) / SLOTHMACHINE / MY SOLACE L...,Club 1019
3,2023-10-05,Ruhail Qaisar (ind) / IDKLANG / SUNDL,rhiz
4,2023-10-05,YOKOHOMO / TUTTI DILEMMA,Chelsea
...,...,...,...
354,2024-06-30,SABBAT (jpn) / GRUESOME (usa),Viper Room
355,2024-10-04,Götz Widmann (d),Arena (kl. Halle)
356,2024-10-05,FAHNENFLUCHT (d),The Loft
357,2024-10-15,BEARTOOTH (usa),Gasometer


## Load latest

In [263]:
df_last = pd.read_csv("capeet_last.csv", parse_dates=["Date"])

df_last

,Date,Event,Location
0,2023-10-05,CHALK HANDS (uk) / CLUTCH AT STRAWS,Arena-Beisl
1,2023-10-05,CHOROSIA / THOSAR / GRIMMS EYE,Viper Room
2,2023-10-05,FISH BASKET (pol) / SLOTHMACHINE / MY SOLACE L...,Club 1019
3,2023-10-05,Ruhail Qaisar (ind) / IDKLANG / SUNDL,rhiz
4,2023-10-05,YOKOHOMO / TUTTI DILEMMA,Chelsea
...,...,...,...
353,2024-06-30,SABBAT (jpn) / GRUESOME (usa),Viper Room
354,2024-10-04,Götz Widmann (d),Arena (kl. Halle)
355,2024-10-05,FAHNENFLUCHT (d),The Loft
356,2024-10-15,BEARTOOTH (usa),Gasometer


## Compare

In [264]:
df_diff = df.merge(
    df_last, on=["Date", "Event", "Location"], how="outer", indicator=True
)

df_diff.loc[df_diff._merge != "both"]

,Date,Event,Location,_merge


## Save

In [265]:
df.to_csv("capeet_last.csv", index=False)